In [7]:
import matplotlib
matplotlib.use('TkAgg')
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
from scipy.interpolate import interp1d
import scipy as sc
import matplotlib.cm as cm
from salishsea_tools import (nc_tools, gsw_calls, geo_tools, viz_tools)
import seabird
import cmocean as cmo
import gsw
from seabird.cnv import fCNV
import pandas as pd
import seaborn as sns
import matplotlib.gridspec as gridspec
import scipy.io
from matplotlib.offsetbox import AnchoredText
from mpl_toolkits.axes_grid1 import make_axes_locatable, axes_size
from eofs.standard import Eof

from dateutil        import parser
from datetime import datetime

import cartopy.crs as ccrs
import cartopy.feature as cfeature

/home/ssahu/anaconda3/lib/python3.5/site-packages/xarray/conventions.py:9: FutureWarning: The pandas.tslib module is deprecated and will be removed in a future version.
  from pandas.tslib import OutOfBoundsDatetime
DEBUG:shapely.geos:Trying `CDLL(libgeos_c.so.1)`
DEBUG:shapely.geos:Trying `CDLL(libgeos_c.so)`
DEBUG:shapely.geos:Trying `CDLL(/home/ssahu/anaconda3/lib/libgeos_c.so)`
DEBUG:shapely.geos:Library path: '/home/ssahu/anaconda3/lib/libgeos_c.so'
DEBUG:shapely.geos:DLL: <CDLL '/home/ssahu/anaconda3/lib/libgeos_c.so', handle 432dd10 at 0x7fa68058a5c0>
DEBUG:shapely.geos:Trying `CDLL(libc.so.6)`
DEBUG:shapely.geos:Library path: 'libc.so.6'
DEBUG:shapely.geos:DLL: <CDLL 'libc.so.6', handle 7fa6c6de3000 at 0x7fa68058a6d8>


In [12]:
mesh_mask_large  = nc.Dataset('/data/mdunphy/NEP036-N30-OUT/INV/mesh_mask.nc')

glamt   = mesh_mask_large.variables['glamt'][:,179:350,479:650]
gphit   = mesh_mask_large.variables['gphit'][:,179:350,479:650]
glamu   = mesh_mask_large.variables['glamu'][:,179:350,479:650]
gphiv   = mesh_mask_large.variables['gphiv'][:,179:350,479:650]
gdepw_0 = mesh_mask_large.variables['gdepw_0'][:,:32,179:350,479:650]
e2u     = mesh_mask_large.variables['e2u'][:,179:350,479:650]
e1v     = mesh_mask_large.variables['e1v'][:,179:350,479:650]
e1t     = mesh_mask_large.variables['e1t'][:,179:350,479:650]
e2t     = mesh_mask_large.variables['e2t'][:,179:350,479:650]
e3t_0   = mesh_mask_large.variables['e3t_0'][:,:32,179:350,479:650]
tmask   = mesh_mask_large.variables['tmask'][:,:32,179:350,479:650]

In [13]:
glamt.shape

(1, 171, 171)

In [14]:
file_mask = nc.Dataset('/data/ssahu/NEP36_2013_summer_hindcast/Ariane_mesh_mask.nc', 'w', zlib=True)

file_mask.createDimension('x', tmask.shape[3]);
file_mask.createDimension('y', tmask.shape[2]);
file_mask.createDimension('z', tmask.shape[1]);
file_mask.createDimension('t', None);

x = file_mask.createVariable('x', 'int32', ('x',), zlib=True);
x.units = 'indices';
x.longname = 'x indices';

y = file_mask.createVariable('y', 'int32', ('y',), zlib=True);
y.units = 'indices';
y.longname = 'y indices';

time_counter = file_mask.createVariable('t', 'int32', ('t',), zlib=True);
time_counter.units = 's';
time_counter.longname = 'time';

glamt_file   = file_mask.createVariable('glamt', 'float32', ('t', 'y', 'x'), zlib=True);
gphit_file   = file_mask.createVariable('gphit', 'float32', ('t', 'y', 'x'), zlib=True);
glamu_file   = file_mask.createVariable('glamu', 'float32', ('t', 'y', 'x'), zlib=True);
gphiv_file   = file_mask.createVariable('gphiv', 'float32', ('t', 'y', 'x'), zlib=True);
e2u_file     = file_mask.createVariable('e2u', 'float32', ('t', 'y', 'x'), zlib=True);
e1v_file     = file_mask.createVariable('e1v', 'float32', ('t', 'y', 'x'), zlib=True);
e1t_file     = file_mask.createVariable('e1t', 'float32', ('t', 'y', 'x'), zlib=True);
e2t_file     = file_mask.createVariable('e2t', 'float32', ('t', 'y', 'x'), zlib=True);
gdepw_0_file = file_mask.createVariable('gdepw_0', 'float32', ('t','z', 'y', 'x'), zlib=True);
e3t_0_file   = file_mask.createVariable('e3t_0', 'float32', ('t','z', 'y', 'x'), zlib=True);
tmask_file   = file_mask.createVariable('tmask', 'float32', ('t','z', 'y', 'x'), zlib=True);

glamt_file[:]   = glamt[:]
gphit_file[:]   = gphit[:]
glamu_file[:]   = glamu[:]
gphiv_file[:]   = gphiv[:]
e2u_file[:]     = e2u[:]
e1v_file[:]     = e1v[:]
e1t_file[:]     = e1t[:]
e2t_file[:]     = e2t[:]
gdepw_0_file[:] = gdepw_0[:]
e3t_0_file[:]   = e3t_0[:]
tmask_file[:]   = tmask[:]
time_counter[0] = 1


file_mask.close()

In [55]:
tmask.shape

(1, 32, 171, 171)

In [5]:
grid_T_small = nc.Dataset('/data/ssahu/NEP36_2013_summer_hindcast/cut_NEP36-S29_1d_20130429_20131025_grid_T_20130429-20130508.nc')

lon_small = grid_T_small.variables['nav_lon'][:]
lat_small = grid_T_small.variables['nav_lat'][:]



In [4]:
lon_A1 = -126.20433
lat_A1 = 48.52958

j, i = geo_tools.find_closest_model_point(lon_A1,lat_A1,\
                                          lon_small,lat_small,grid='NEMO',tols=\
                                          {'NEMO': {'tol_lon': 0.1, 'tol_lat': 0.1},\
                                           'GEM2.5': {'tol_lon': 0.1, 'tol_lat': 0.1}}) 

print(j,i)

90 93


In [56]:
tmask.shape[3]

171

glamt_large.shape

In [15]:
lon_small[0,0], lat_small[0,0]

(-128.85672, 47.343193)

In [16]:
lon_small[-1,0], lat_small[-1,0]

(-128.54237, 49.788475)

In [17]:
lon_small[0,-1], lat_small[0,-1]

(-124.24197, 47.136707)

In [18]:
lon_small[-1,-1], lat_small[-1,-1]

(-123.98723, 49.517643)

In [11]:
mat_file_str='/data/ssahu/Falkor_2013/mvp/surveyA.mat'



mat = scipy.io.loadmat(mat_file_str)

depths_survey = mat['depths'][:,0]
lat_survey = mat['latitude'][:,0]
lon_survey = mat['longitude'][:,0] - 100
# den_survey = mat['density'][:]
pden_survey = mat['pden'][:]
temp_survey = mat['temp'][:]
sal_survey = mat['salinity'][:]
mtime = mat['mtime'][:,0]

pressure_survey = np.empty_like(temp_survey)
SA_survey = np.empty_like(temp_survey)
CT_survey = np.empty_like(temp_survey)
spic_survey = np.empty_like(temp_survey)
rho_survey = np.empty_like(temp_survey)


y = np.empty_like(lat_survey)
x = np.empty_like(y)


for i in np.arange(lat_survey.shape[0]):
    y[i], x[i] = geo_tools.find_closest_model_point(
               lon_survey[i],lat_survey[i],lon_small,lat_small,tols={
        'NEMO': {'tol_lon': 0.1, 'tol_lat': 0.1},'GEM2.5': {'tol_lon': 0.1, 'tol_lat': 0.1}})



In [12]:
for i in np.arange(y.shape[0]):
    print (y[i], x[i])

97.0 103.0
97.0 103.0
97.0 103.0
97.0 103.0
97.0 103.0
95.0 103.0
95.0 103.0
95.0 103.0
94.0 103.0
94.0 103.0
94.0 103.0
93.0 103.0
93.0 102.0
93.0 102.0
92.0 102.0
92.0 102.0
92.0 102.0
91.0 102.0
91.0 102.0
90.0 102.0
90.0 102.0
89.0 102.0
89.0 102.0
89.0 102.0
88.0 101.0
87.0 101.0
87.0 101.0
85.0 101.0
85.0 101.0
84.0 101.0
83.0 100.0
82.0 100.0
81.0 100.0
80.0 100.0
80.0 101.0
80.0 101.0
81.0 102.0
81.0 102.0
82.0 103.0
82.0 103.0
83.0 104.0
83.0 105.0
84.0 105.0
84.0 105.0
85.0 106.0
85.0 106.0
85.0 107.0
86.0 107.0
86.0 107.0
86.0 107.0
86.0 108.0
87.0 108.0
87.0 108.0
87.0 109.0
87.0 109.0
88.0 109.0
88.0 109.0
88.0 110.0
88.0 110.0
89.0 110.0
89.0 110.0
89.0 111.0
89.0 111.0
90.0 111.0
90.0 111.0
90.0 112.0
90.0 112.0
89.0 112.0
89.0 112.0
89.0 112.0
88.0 112.0
88.0 112.0
88.0 111.0
87.0 111.0
87.0 111.0
87.0 111.0
86.0 111.0
86.0 111.0
85.0 111.0
85.0 110.0
84.0 110.0
84.0 110.0
84.0 110.0
83.0 110.0
83.0 109.0
82.0 109.0
82.0 109.0
81.0 109.0
81.0 109.0
80.0 108.0
80.0 108.0